In [82]:
import imageio.v3 as imageio
import torch
import torch.nn as nn
import os
from glob import glob
import numpy as np
import pandas as pd

In [75]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# Imágenes

- *Hay muchas formas de codificar los colores de una imagen. La forma más común es utilizar el método RGB, donde cada pixel es una combinación de tres colores, donde cada color representa la intensidad de rojo, verde y azul, respectivamente.*
<figure>
    <img src='attachments/color-image-tensor.png' width=1000 style="display: block; margin: 0 auto;" />
    <figcaption style="text-align: center;"> Stevens, Eli. (2020). A rainbow image, broken into red, blue, and green channels. In Stevens, Eli, <i>Deep Learning with PyTorch</i> (p. 72).
    </figcaption>
</figure>

In [8]:
image = imageio.imread('../data/fundamentals/image-dog/bobby.jpg')
image.shape

(720, 1280, 3)

- *Notar que las primeras dos dimensiones corresponden a la altura y el ancho de la imagen, mientras que la tercera dimensión corresponde a los canales.*
- *Los modulos de PyTorch que trabajan con imágenes requieren que la primera dimensión del tensor sean los canales. Corregimos las dimensiones del tensor con la función `torch.Tensor.permute()`, donde especificamos que la primera dimensión ahora sea la `dim=2` del tensor original.*

In [ ]:
img = torch.from_numpy(image).to(device=device)
print(f'Original tensor:\n {img}')
print(f'\nOriginal shape: {img.shape}')

Original tensor:
 tensor([[[ 77,  45,  22],
         [ 77,  45,  22],
         [ 78,  46,  21],
         ...,
         [118,  78,  52],
         [117,  77,  51],
         [116,  76,  51]],

        [[ 75,  43,  20],
         [ 76,  44,  21],
         [ 77,  45,  20],
         ...,
         [118,  78,  52],
         [117,  77,  51],
         [116,  76,  50]],

        [[ 74,  39,  17],
         [ 75,  41,  16],
         [ 77,  43,  18],
         ...,
         [119,  80,  51],
         [117,  77,  51],
         [116,  76,  50]],

        ...,

        [[215, 165,  78],
         [216, 166,  79],
         [217, 167,  80],
         ...,
         [172, 122,  51],
         [174, 124,  53],
         [174, 124,  53]],

        [[215, 165,  78],
         [216, 166,  79],
         [217, 167,  80],
         ...,
         [173, 123,  54],
         [174, 124,  55],
         [174, 124,  55]],

        [[215, 165,  78],
         [216, 166,  79],
         [217, 167,  80],
         ...,
         [159, 1

In [10]:
img_permuted = img.permute(2, 0, 1)
print(f'Permuted tensor:\n {img_permuted}')
print(f'\nPermuted shape: {img_permuted.shape}')

Permuted tensor:
 tensor([[[ 77,  77,  78,  ..., 118, 117, 116],
         [ 75,  76,  77,  ..., 118, 117, 116],
         [ 74,  75,  77,  ..., 119, 117, 116],
         ...,
         [215, 216, 217,  ..., 172, 174, 174],
         [215, 216, 217,  ..., 173, 174, 174],
         [215, 216, 217,  ..., 159, 158, 158]],

        [[ 45,  45,  46,  ...,  78,  77,  76],
         [ 43,  44,  45,  ...,  78,  77,  76],
         [ 39,  41,  43,  ...,  80,  77,  76],
         ...,
         [165, 166, 167,  ..., 122, 124, 124],
         [165, 166, 167,  ..., 123, 124, 124],
         [165, 166, 167,  ..., 108, 107, 107]],

        [[ 22,  22,  21,  ...,  52,  51,  51],
         [ 20,  21,  20,  ...,  52,  51,  50],
         [ 17,  16,  18,  ...,  51,  51,  50],
         ...,
         [ 78,  79,  80,  ...,  51,  53,  53],
         [ 78,  79,  80,  ...,  54,  55,  55],
         [ 78,  79,  80,  ...,  42,  41,  41]]], dtype=torch.uint8)

Permuted shape: torch.Size([3, 720, 1280])


- *El nuevo tensor `img_permuted` es una vista del objeto en memoria que se creó cuando definimos el tensor original (`img`).* 
- *Ambos tensores apuntan al mismo objecto en memoria, la única diferencia entre los tensores es el stride que utilizan para moverse a lo largo del objeto `torch.Storage`.*
- *Para entrenar una red neuronal vamos a utilizar más de una imagen. Podemos representar un batch de imágenes como un tensor de cuatro dimensiones, donde la primera dimensión corresponde a la cantidad de imágenes que tenemos en el batch, la segunda dimensión corresponde a los canales, y las últimas dos corresponden a la altura y el ancho de cada imagen.*

In [ ]:
batch = torch.zeros(3, 3, 256, 256, dtype=torch.uint8)

directory = '../data/fundamentals/image-cats/'
images = [image for image in os.listdir(directory) if image.endswith('.png')]
for (index, image) in enumerate(images):
    # Loads image
    img = imageio.imread(os.path.join(directory, image))

    # Converts the array to a tensor and permutes the dimensions
    img = torch.from_numpy(img).to(device=device)
    img = img.permute(2, 0, 1)

    # Adds the image to the batch
    batch[index] = img

    print(f'Added {image} to batch')

Added cat1.png to batch
Added cat2.png to batch
Added cat3.png to batch


In [46]:
batch

tensor([[[[156, 152, 124,  ..., 150, 149, 158],
          [174, 134, 165,  ..., 120, 136, 138],
          [127, 156, 107,  ..., 131, 143, 164],
          ...,
          [116, 130, 129,  ..., 127, 118, 112],
          [129, 130, 123,  ..., 115, 121, 114],
          [129, 123, 118,  ..., 113, 121, 120]],

         [[139, 135, 109,  ..., 135, 135, 147],
          [160, 119, 149,  ..., 105, 122, 124],
          [113, 140,  90,  ..., 118, 129, 152],
          ...,
          [ 99, 110, 111,  ..., 117, 108, 103],
          [111, 111, 106,  ..., 106, 112, 105],
          [111, 104, 102,  ..., 103, 110, 111]],

         [[129, 123,  98,  ..., 131, 132, 145],
          [155, 110, 137,  ..., 102, 119, 121],
          [104, 132,  80,  ..., 112, 125, 146],
          ...,
          [ 93, 108, 105,  ..., 125, 115, 108],
          [108, 108,  98,  ..., 110, 117, 110],
          [107,  98,  95,  ..., 108, 115, 116]]],


        [[[202, 193, 190,  ...,  13,  13,  12],
          [199, 192, 189,  ...,  14

## Normalizando los datos

- *Las redes neuronales suelen performar mejor cuando los inputs se encuentran en el rango $[0, 1]$ (o $[-1, 1]$). Usualmente vamos a querer normalizar nuestros datos, restando la media y dividiendo por el desvío estándar, a lo largo de cada uno de los canales.*
- ***IMPORTANTE.** En la práctica, cuando se cuenta con un dataset completo, se suele calcular primero la media y el desvío estándar para cada canal sobre todo el dataset, y luego normalizar por batches.*

In [ ]:
# Convert the batch to float
batch_norm = batch.clone().to(dtype=torch.float32)

# Iterate over the channels to compute the mean and standard deviation, and then normalize
n_channels = batch_norm.shape[1]
for channel in np.arange(n_channels):
    # Compute the mean and standard deviation for the current channel
    mean = batch_norm[:, channel].mean()
    std = batch_norm[:, channel].std()

    # Normalize the current channel
    batch_norm[:, channel] = (batch[:, channel] - mean) / std

In [51]:
batch_norm

tensor([[[[ 0.1439,  0.0730, -0.4234,  ...,  0.0375,  0.0198,  0.1794],
          [ 0.4631, -0.2461,  0.3035,  ..., -0.4944, -0.2107, -0.1752],
          [-0.3703,  0.1439, -0.7249,  ..., -0.2993, -0.0866,  0.2858],
          ...,
          [-0.5653, -0.3171, -0.3348,  ..., -0.3703, -0.5298, -0.6362],
          [-0.3348, -0.3171, -0.4412,  ..., -0.5830, -0.4766, -0.6007],
          [-0.3348, -0.4412, -0.5298,  ..., -0.6185, -0.4766, -0.4944]],

         [[ 0.4632,  0.3874, -0.1058,  ...,  0.3874,  0.3874,  0.6150],
          [ 0.8615,  0.0839,  0.6529,  ..., -0.1816,  0.1408,  0.1787],
          [-0.0299,  0.4822, -0.4661,  ...,  0.0649,  0.2736,  0.7098],
          ...,
          [-0.2954, -0.0868, -0.0678,  ...,  0.0460, -0.1247, -0.2196],
          [-0.0678, -0.0678, -0.1627,  ..., -0.1627, -0.0489, -0.1816],
          [-0.0678, -0.2006, -0.2385,  ..., -0.2196, -0.0868, -0.0678]],

         [[ 0.7792,  0.6573,  0.1495,  ...,  0.8198,  0.8401,  1.1041],
          [ 1.3072,  0.3933,  

# Imágenes médicas

- *Las imágenes médicas, como las de una tomografía computada, están compuestas por capas a lo largo de tres dimensiones: altura, ancho y profundidad.* 
- *La profundidad hace referencia a la distancia en el cuerpo del paciente. Por ejemplo, si se realiza una tomografía del torso de un paciente, capturando 200 imágenes con 1mm de distancia entre una y otra, desde el pecho hasta el abdomen, la profundidad de la imagen representa 200mm de espacio físico sobre cuerpo del paciente.*
<figure>
    <img src='attachments/medical-image-tensor.png' width=1000 style="display: block; margin: 0 auto;" />
    <figcaption style="text-align: center;"> Stevens, Eli. (2020). Slices of a CT scan, from the top of the head to the jawline. In Stevens, Eli, <i>Deep Learning with PyTorch</i> (p. 72).
    </figcaption>
</figure>

- *Las imágenes médicas tienen un único canal de colores, con una sola intesidad (blanco y negro). Entonces, para representar una imagen médica, necesitamos cuatro dimensiones: el canal, la profundidad, la altura y el ancho.*

In [ ]:
# Set the directory containing the DICOM files
directory = '../data/fundamentals/volume-data'

# Get a list of all DICOM files in the directory
files = sorted(glob(os.path.join(directory, '*.dcm')))
print(f'Found {len(files)} files\n')

# Loads the DICOM files into a list. Each element is a 2D array that represents a slice of the volume
dicom_files = [imageio.imread(file, plugin='DICOM') for file in files]

# Create a 3D array and create a tensor
volume_arr = np.stack(dicom_files, axis=0)
print(f'\nVolume shape: {volume_arr.shape}')

# Convert the volume to a tensor
volume = torch.from_numpy(volume_arr).to(device=device)
print(f'\nVolume tensor shape: {volume.shape}')

Found 99 files

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining

- *Todavía falta incluir el canal en nuestro tensor. Utilizamos la función `torch.unsqueeze()`, que nos permite agregar una dimensión de tamaño uno.*

In [66]:
volume = torch.unsqueeze(volume, 0)
print(f'Volume tensor shape after unsqueeze: {volume.shape}')

Volume tensor shape after unsqueeze: torch.Size([1, 99, 512, 512])


# Datos tabulares

- *Los datos tabulares son, típicamente, no homogéneos. Esto quiere decir que nos podemos encontrar con una columna que tenga datos númericos, otra que tenga datos booleanos, y otra que tenga texto.*
- *Por otro lado, los tensores de PyTorch son homogéneos. La información se encuentra codificada en números, ya que las redes neuronales son entidades matemáticas que reciben números como inputs y devuelven números como output.*

In [ ]:
# Loads dataset using Pandas
df = pd.read_parquet('../data/fundamentals/tabular-data/winequality-white.parquet')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [ ]:
# Convert the data to a NumPy array
wine_array = df.to_numpy()
wine_array

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], shape=(4898, 12))

In [77]:
tensor = torch.from_numpy(wine_array).to(dtype=torch.float32, device='mps')
print(f'Tensor data:\n {tensor}')
print(f'Tensor shape: {tensor.shape}')

Tensor data:
 tensor([[ 7.0000,  0.2700,  0.3600,  ...,  0.4500,  8.8000,  6.0000],
        [ 6.3000,  0.3000,  0.3400,  ...,  0.4900,  9.5000,  6.0000],
        [ 8.1000,  0.2800,  0.4000,  ...,  0.4400, 10.1000,  6.0000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  0.4600,  9.4000,  6.0000],
        [ 5.5000,  0.2900,  0.3000,  ...,  0.3800, 12.8000,  7.0000],
        [ 6.0000,  0.2100,  0.3800,  ...,  0.3200, 11.8000,  6.0000]],
       device='mps:0')
Tensor shape: torch.Size([4898, 12])


In [ ]:
# Separate the features and the target
X = tensor[:, :-1]
y = tensor[:, -1]